## Data Wrangling with Python: Intro to Pandas
Note: Notebook adapted from [here](https://github.com/EricElmoznino/lighthouse_pandas_tutorial/blob/master/pandas_tutorial.ipynb) & [here](https://github.com/sedv8808/LighthouseLabs/tree/main/W02D2) & from LHL's [21 Day Data Challenge](https://data-challenge.lighthouselabs.ca/start)
#### Instructor: Simon Dawkins
**Agenda:**
 - Why Pandas?
 - Pandas Basics
     - Pandas Series vs. Pandas DataFrames
     - .loc() vs. iloc()
 - Pandas Advance
     - Filtering
     - Group-bys
 - Pandas Exercises
     - Challenge 1
     - Challenge 2

### Pandas: Why Pandas? What is it? 


To do data anlaysis with Python, Pandas is a great tool to for dealing with data in a tabular and time series formats. Designed by Wes McKinney as an attempt to port R's dataframes to python. 

- Python Package for working with **tables**
- Similar to SQL & Excel
    - Faster sometimes - more optimized for non-distributed platforms
    - More features to manipulate, transform, and aggregate data
- Easy to handle messy and missing data
- Great at working with large datasets
- When combing with other Python libraries, it's fairly easy to create beautiful and customazied visuals. Easy integration with Matplotlib, Seaborn, Plotly.
- Easy integration with machine learning plugins (sckit-learn, etc)
    
    
-----------
To read more about, Wes McKinney, the creator of Pandas, check out the article below.

1. https://qz.com/1126615/the-story-of-the-most-important-tool-in-data-science/

--------------


## Think of how we would try to represent a table in Python?


In [2]:
#A dicitonary of lists example
students = {
    'student_id': [1, 2, 3, 4,5,6],
    'name': ['Daenerys', 'Jon', 'Arya', 'Sansa', 'Eddard', 'Khal Drogo'],
    'course_mark': [82, 100, 12, 76, 46, 20],
    'species': ['cat', 'human', 'cat', 'human', 'human', 'human']
}

**What are some operations we might want to do on this data?**

- 1. Select a subset of columns
- 2. Filter out some rows based on an attribute
- 3. Group by some attribute
- 4. Compute some aggregate values within groups
- 5. Save to a file

How about we try out one of these to see how easy it is

### Return a table with the mean course mark per-species with vanilla Python

In [3]:
# Return a table with the mean course mark per species

species_sums = {} # Tables of Sums
species_counts = {} # Count per Species
for i in range(len(students['species'])):  # iterating over the rows
    species = students['species'][i] # every row number I get species 
    course_mark = students['course_mark'][i] #  and course mark
    if species not in species_sums: # Intializing Species if not in list
        species_sums[species] = 0
        species_counts[species] = 0
    species_sums[species] += course_mark # Add each course mark for each species
    species_counts[species] += 1 

species_means = {}
                                  
for species in species_sums: # for every unique species we found
    species_means[species] = species_sums[species] / species_counts[species] #sum/count

species_means

{'cat': 47.0, 'human': 60.5}

- Did you like looking at this? 
- Does this look fun to do? 
- what about with 10000 rows and 50 columns?
- Super Tiring.

## Pandas Version

In [4]:
import pandas as pd

# Can take in a dictionry of list to instantiate a DataFrame
students = pd.DataFrame(students)
students

,student_id,name,course_mark,species
0,1,Daenerys,82,cat
1,2,Jon,100,human
2,3,Arya,12,cat
3,4,Sansa,76,human
4,5,Eddard,46,human
5,6,Khal Drogo,20,human


In [5]:
species_means = students[['species', 'course_mark']].groupby('species').mean()
# species_means = students.groupby('species')['course_mark'].mean()
species_means

,course_mark
species,
cat,47.0
human,60.5


### Dissecting the above code:


In [6]:
#Step 1: Filter out the columns we want to keep
students_filtered = students[['species','course_mark']]
students_filtered

,species,course_mark
0,cat,82
1,human,100
2,cat,12
3,human,76
4,human,46
5,human,20


In [8]:
# Step 2: Group by species column
students_grouped_by_species = students_filtered.groupby('species') 
students_grouped_by_species # only does something after calling an aggregate function on groups

In [9]:
#Step 3: Specify how to aggregate the course-mark column
species_means = students_grouped_by_species.mean()

In [10]:
species_means

,course_mark
species,
cat,47.0
human,60.5


#### As shown, Pandas makes use of vectorized operations. 


- Rather than use for-loops, we specify the operation that will apply to the structure as a whole (i.e. all the rows)
- By vectorizing, **the code becomes more concise and more readable**
- Pandas is optimized for vectorized operations (parallel vs. serial computation), which makes them **much faster**
- It is almost always possible to vectorize operations on Pandas data types - series have a single data type


### Getting Started: Pandas Series & Pandas DataFrames

There are two Pandas data types of interest:

- Series (column)
    - A pandas series is similar to an array but it has an index. The index is constant, and doesnt change through the operations we apply to the series. 
- DataFrame (table)
    - A pandas dataframe is an object that is similar to a collection of pandas series.

In [12]:
# One way to construct a Series
grades = pd.Series([82, 100, 12, 76, 46, 20]) # has to all be the same data type
grades

0     82
1    100
2     12
3     76
4     46
5     20
dtype: int64

In [16]:
#We can specify some index when building a series. 
grades = pd.Series([82, 100, 12, 76, 46, 20], 
                   index = ['Daenerys', 'Jon', 'Arya', 'Sansa', 'Eddard', 'Khal Drogo']) 

grades

Daenerys       82
Jon           100
Arya           12
Sansa          76
Eddard         46
Khal Drogo     20
dtype: int64

In [17]:
print("The values:", grades.values)
print("The indexes:", grades.index)

The values: [ 82 100  12  76  46  20]
The indexes: Index(['Daenerys', 'Jon', 'Arya', 'Sansa', 'Eddard', 'Khal Drogo'], dtype='object')


**Note:** The underlying index is still 0, 1, 2, 3.... and we can still index on that:

In [22]:
grades[2] # can use the index number

12

In [23]:
grades['Arya'] # or the name of the index

12

### Pandas DataFrames

In [24]:
# One way to construct a DataFrame
df = pd.DataFrame({
    'name': ['Daenerys', 'Jon', 'Arya', 'Sansa'],
    'course_mark': [82, 100, 12, 76],
    'species': ['human', 'human', 'cat', 'human']},
    index=[1412, 94, 9351, 14])
df

,name,course_mark,species
1412,Daenerys,82,human
94,Jon,100,human
9351,Arya,12,cat
14,Sansa,76,human


#### Reading a CSV file

We'll use the function `read_csv()` to load the data into our notebook

- The `read_csv()` function can read data from a locally saved file or from a URL
- We'll store the data as a variable `df_pokemon`

In [25]:
pokemon = pd.read_csv('pokemon.csv')

In [26]:
pokemon

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


**What do we see here?**
- Each row of the table is an observation, containing data of a single pokemon

In [29]:
# shape

pokemon.shape

(800, 13)

For large DataFrames, it's often useful to display just the first few or last few rows:

In [30]:
# head
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [53]:
# tail
pokemon.tail()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True
799,721,Volcanion,Fire,Water,600,80,110,120,130,90,70,6,True


## Data at a Glance

`pandas` provides many ways to quickly and easily summarize your data:
- How many rows and columns are there?
- What are all the column names and what type of data is in each column?
- How many values are missing in each column or row?
- Numerical data: What is the average and range of the values?
- Text data: What are the unique values and how often does each occur?

### Peeking into the pokemon dataset

- Similar with getting familar with SQL tables, it is often a great idea to look at the pandas dataframes we are working with. Below are some of the basic methods to glance at a dataset. 

In [31]:
# Getting the Columns
pokemon.columns

Index(['#', 'Name', 'Type 1', 'Type 2', 'Total', 'HP', 'Attack', 'Defense',
       'Sp. Atk', 'Sp. Def', 'Speed', 'Generation', 'Legendary'],
      dtype='object')

In [32]:
# Getting Summary Statistics
pokemon.describe()

,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
count,800.000000,800.00000,800.000000,800.000000,800.000000,800.000000,800.000000,800.000000,800.00000
mean,362.813750,435.10250,69.258750,79.001250,73.842500,72.820000,71.902500,68.277500,3.32375
std,208.343798,119.96304,25.534669,32.457366,31.183501,32.722294,27.828916,29.060474,1.66129
min,1.000000,180.00000,1.000000,5.000000,5.000000,10.000000,20.000000,5.000000,1.00000
25%,184.750000,330.00000,50.000000,55.000000,50.000000,49.750000,50.000000,45.000000,2.00000
50%,364.500000,450.00000,65.000000,75.000000,70.000000,65.000000,70.000000,65.000000,3.00000
75%,539.250000,515.00000,80.000000,100.000000,90.000000,95.000000,90.000000,90.000000,5.00000
max,721.000000,780.00000,255.000000,190.000000,230.000000,194.000000,230.000000,180.000000,6.00000


In [34]:
# accessing columns with dot
pokemon.Total.describe()

count    800.00000
mean     435.10250
std      119.96304
min      180.00000
25%      330.00000
50%      450.00000
75%      515.00000
max      780.00000
Name: Total, dtype: float64

In [36]:
pokemon[['Total','HP']].describe() # better to use square brackets and string - can access several columns, can access columns with spaces in the name

,Total,HP
count,800.00000,800.000000
mean,435.10250,69.258750
std,119.96304,25.534669
min,180.00000,1.000000
25%,330.00000,50.000000
50%,450.00000,65.000000
75%,515.00000,80.000000
max,780.00000,255.000000


In [39]:
# Checking for Missing Data
pokemon.isnull().sum() / len(pokemon)

#             0.0000
Name          0.0000
Type 1        0.0000
Type 2        0.4825
Total         0.0000
HP            0.0000
Attack        0.0000
Defense       0.0000
Sp. Atk       0.0000
Sp. Def       0.0000
Speed         0.0000
Generation    0.0000
Legendary     0.0000
dtype: float64

In [40]:
# getting info
pokemon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


## The .loc() vs .iloc() method


To select rows and columns at the same time, we use the syntax `.loc[ROWS SLICE, COLUMNS SLICE]`:

In [44]:
# simple slice
# Notice the square brackets on loc
pokemon.loc[20:30,'Attack'] ## note that start and stop are inclusive

20    45
21    60
22    80
23    80
24    56
25    81
26    60
27    90
28    60
29    85
30    55
Name: Attack, dtype: int64

In [47]:
# Getting more than one column
pokemon.loc[0:40:3, ['Name','Attack']]

,Name,Attack
0,Bulbasaur,49
3,VenusaurMega Venusaur,100
6,Charizard,84
9,Squirtle,48
12,BlastoiseMega Blastoise,103
15,Butterfree,45
18,Beedrill,90
21,Pidgeotto,60
24,Rattata,56
27,Fearow,90


In [48]:
# pass a list for the rows
pokemon.loc[[3,5,66,1],['Name','Legendary']]

,Name,Legendary
3,VenusaurMega Venusaur,False
5,Charmeleon,False
66,Poliwhirl,False
1,Ivysaur,False


In [50]:
# pass a range range of column values
pokemon.loc[10:18, 'Name':'Type 2']

,Name,Type 1,Type 2
10,Wartortle,Water,NaN
11,Blastoise,Water,NaN
12,BlastoiseMega Blastoise,Water,NaN
13,Caterpie,Bug,NaN
14,Metapod,Bug,NaN
15,Butterfree,Bug,Flying
16,Weedle,Bug,Poison
17,Kakuna,Bug,Poison
18,Beedrill,Bug,Poison


In [52]:
#iloc is used for integer based indexing
pokemon.iloc[2:50:4, 0:6:2]

,#,Type 1,Total
2,3,Grass,525
6,6,Fire,534
10,8,Water,405
14,11,Bug,205
18,15,Bug,395
22,18,Normal,479
26,21,Normal,262
30,25,Electric,320
34,29,Poison,275
38,33,Poison,365


### Modifying a Column or Creating a new column

In [54]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [56]:
# Make sure make a copy of your dataframe before doing any modification
# This is generaly best practice, and it will be easier for you to keep track of the various
# changes and states of your dataframe. 
pokemon_2 = pokemon.copy()
pokemon_2

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


In [57]:
# create a new column
# Combine Attack + Special Attack
pokemon_2['Total Attack'] = pokemon_2['Attack'] + pokemon_2['Sp. Atk']
pokemon_2

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Total Attack
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False,114
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False,142
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False,182
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False,222
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True,200
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True,320
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True,260
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True,330


In [60]:
# modify existing column
pokemon_2['Total'] = pokemon['Total']*2
pokemon_2

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Total Attack
0,1,Bulbasaur,Grass,Poison,636,45,49,49,65,65,45,1,False,114
1,2,Ivysaur,Grass,Poison,810,60,62,63,80,80,60,1,False,142
2,3,Venusaur,Grass,Poison,1050,80,82,83,100,100,80,1,False,182
3,3,VenusaurMega Venusaur,Grass,Poison,1250,80,100,123,122,120,80,1,False,222
4,4,Charmander,Fire,NaN,618,39,52,43,60,50,65,1,False,112
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,1200,50,100,150,100,150,50,6,True,200
796,719,DiancieMega Diancie,Rock,Fairy,1400,50,160,110,160,110,110,6,True,320
797,720,HoopaHoopa Confined,Psychic,Ghost,1200,80,110,60,150,130,70,6,True,260
798,720,HoopaHoopa Unbound,Psychic,Dark,1360,80,160,60,170,130,80,6,True,330


In [68]:
# Modify Data Frame with .loc() method
pokemon_2.loc[:50, 'Name'] = 'Simon'

### Sort_values() & value_counts()

1. ***df.sort_values()***
2. ***df.value_counts()***


The ***pandas.sort_values()*** allows us to reorder our dataframe in an ascending or descending order given a column for pandas to work from. This is similar to the excel sort function.

```python
import pandas as pd
df = pd.read_csv('random.csv')
df


df.sort_values(by=['some_column'], ascending = True)
```
In the above code snippet, we are sorting our *random.csv* pandas data frame by the column *some_column* in ascending order. To read more on the ***df.sort_values()*** function, read this [article](https://datatofish.com/sort-pandas-dataframe/).

The second function is ***df.value_counts()***, it allows us to count how many times a specific value/item occurred in the dataframe. This function is best used on a specific column on a data frame, ideally on a column representing categorical data. Categorical data refers to a statistical data type consisting of categorical variables. 

```python
df['column'].value_counts()
```

To read more on some of the advanced functionalities of ***df.value_counts()***, please refer to the pandas documentation or this [article](https://towardsdatascience.com/getting-more-value-from-the-pandas-value-counts-aa17230907a6).

In [69]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [73]:
# list Unique Values
pokemon.sort_values(by='Attack', ascending = False)
pokemon.sort_values('Attack') # don't need by if only sorting by one column
pokemon.sort_values(by = ['Attack', 'HP'], ascending = [True,False])

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
121,113,Chansey,Normal,NaN,450,250,5,5,35,105,50,1,False
488,440,Happiny,Normal,NaN,220,100,5,5,15,65,30,4,False
261,242,Blissey,Normal,NaN,540,255,10,10,75,135,55,2,False
139,129,Magikarp,Water,NaN,200,20,10,55,15,20,80,1,False
230,213,Shuckle,Bug,Rock,505,20,10,230,10,230,5,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
426,384,RayquazaMega Rayquaza,Dragon,Flying,780,105,180,100,180,100,115,3,True
424,383,GroudonPrimal Groudon,Ground,Fire,770,100,180,160,150,90,90,3,True
429,386,DeoxysAttack Forme,Psychic,NaN,600,50,180,20,180,20,150,3,True
232,214,HeracrossMega Heracross,Bug,Fighting,600,80,185,115,40,105,75,2,False


In [75]:
pokemon['Type 1'].value_counts() # meant for categorical columns

Water       112
Normal       98
Grass        70
Bug          69
Psychic      57
Fire         52
Electric     44
Rock         44
Dragon       32
Ground       32
Ghost        32
Dark         31
Poison       28
Steel        27
Fighting     27
Ice          24
Fairy        17
Flying        4
Name: Type 1, dtype: int64

In [76]:
pokemon['Type 1'].value_counts().sort_index()

Bug          69
Dark         31
Dragon       32
Electric     44
Fairy        17
Fighting     27
Fire         52
Flying        4
Ghost        32
Grass        70
Ground       32
Ice          24
Normal       98
Poison       28
Psychic      57
Rock         44
Steel        27
Water       112
Name: Type 1, dtype: int64

In [77]:
# How many unique Values
pokemon['Type 1'].unique()

array(['Grass', 'Fire', 'Water', 'Bug', 'Normal', 'Poison', 'Electric',
       'Ground', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice',
       'Dragon', 'Dark', 'Steel', 'Flying'], dtype=object)

In [79]:
pokemon['Type 2'].nunique()

18

### How to Query or Filter Data with Conditions?

- We can extract specific data from our dataframe based on a specific condition. We will be using the syntax below. Pandas will return a subset of the dataframe based on the given condition. 

```python
df[<insert_condition>]
```

Conditions follow the generic boolean logic in Python. Below is a cheat sheet python boolean logic.

**Conditional Logic:** 

Conditional logic refers to the execution of different actions based on whether a certain condition is met. In programming, these conditions are expressed by a set of symbols called **Boolean Operators**. 

| Boolean Comparator | Example | Meaning                         |
|--------------------|---------|---------------------------------|
| >                  | x > y   | x is greater than y             |
| >=                 | x >= y  | x is greater than or equal to y |
| <                  | x < y   | x is less than y                |
| <=                 | x <= y  | x is less than or equal to y    |
| !=                 | x != y  | x is not equal to y             |
| ==                 | x == y  | x is equal to y                 |




In [84]:
# Step 1: Create a filter
the_filter = pokemon['Total'] >= 500

In [85]:
# Step 2: Apply Filter
pokemon[the_filter]
pokemon[pokemon['Total'] >= 500]

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
6,6,Charizard,Fire,Flying,534,78,84,78,109,85,100,1,False
7,6,CharizardMega Charizard X,Fire,Dragon,634,78,130,111,130,85,100,1,False
8,6,CharizardMega Charizard Y,Fire,Flying,634,78,104,78,159,115,100,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


Alternativly we can use the pandas **.where()** function, which has the following syntax.


```python 
df.where(<condition>, <What to fill inplace where the condition is not True>) # default is Nan
```

In [83]:
pokemon.where(the_filter, 'Strong Pokemon')

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon
1,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon,Strong Pokemon
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


### Grouping and Aggregation 

Grouping and aggregation can be used to calculate statistics on groups in the data.

**Common Aggregation Functions**
- mean()
- median()
- sum()
- count()


In [86]:
# simple groupby
pokemon.groupby('Type 1').mean()

C:\Users\User\AppData\Local\Temp\ipykernel_5236\1829787788.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pokemon.groupby('Type 1').mean()


,#,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
Type 1,,,,,,,,,,
Bug,334.492754,378.927536,56.884058,70.971014,70.724638,53.869565,64.797101,61.681159,3.217391,0.000000
Dark,461.354839,445.741935,66.806452,88.387097,70.225806,74.645161,69.516129,76.161290,4.032258,0.064516
Dragon,474.375000,550.531250,83.312500,112.125000,86.375000,96.843750,88.843750,83.031250,3.875000,0.375000
Electric,363.500000,443.409091,59.795455,69.090909,66.295455,90.022727,73.704545,84.500000,3.272727,0.090909
Fairy,449.529412,413.176471,74.117647,61.529412,65.705882,78.529412,84.705882,48.588235,4.117647,0.058824
Fighting,363.851852,416.444444,69.851852,96.777778,65.925926,53.111111,64.703704,66.074074,3.370370,0.000000
Fire,327.403846,458.076923,69.903846,84.769231,67.769231,88.980769,72.211538,74.442308,3.211538,0.096154
Flying,677.750000,485.000000,70.750000,78.750000,66.250000,94.250000,72.500000,102.500000,5.500000,0.500000
Ghost,486.500000,439.562500,64.437500,73.781250,81.187500,79.343750,76.468750,64.343750,4.187500,0.062500


- By default, `groupby()` assigns the variable that we're grouping on (in this case `Type 1`) to the index of the output data
- If we use the keyword argument `as_index=False`, the grouping variable is instead assigned to a regular column
  - This can be useful in some situations, such as data visualization functions which expect the relevant variables to be in columns rather than the index

In [88]:
# grouby with as_index set to False
pokemon.groupby('Type 1', as_index=False)['HP'].max()

,Type 1,HP
0,Bug,86
1,Dark,126
2,Dragon,125
3,Electric,90
4,Fairy,126
5,Fighting,144
6,Fire,115
7,Flying,85
8,Ghost,150
9,Grass,123


We can use the `agg` method to compute multiple aggregate functions on our data, for example minimum, maximum and mean attack in Type 1

In [90]:
# combine multiple agg functions 
pokemon.groupby(['Type 1', 'Legendary']).mean()

pokemon.groupby('Type 1').agg(['min', 'max', 'mean'])

C:\Users\User\AppData\Local\Temp\ipykernel_5236\2617560061.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  pokemon.groupby(['Type 1', 'Legendary']).mean()
C:\Users\User\AppData\Local\Temp\ipykernel_5236\2617560061.py:4: FutureWarning: ['Name', 'Type 2'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  pokemon.groupby('Type 1').agg(['min', 'max', 'mean'])


#                  Total                   HP                  \
          min  max        mean   min  max        mean min  max       mean   
Type 1                                                                      
Bug        10  666  334.492754   194  600  378.927536   1   86  56.884058   
Dark      197  717  461.354839   220  680  445.741935  35  126  66.806452   
Dragon    147  718  474.375000   300  780  550.531250  41  125  83.312500   
Electric   25  702  363.500000   205  610  443.409091  20   90  59.795455   
Fairy      35  716  449.529412   218  680  413.176471  35  126  74.117647   
Fighting   56  701  363.851852   210  625  416.444444  30  144  69.851852   
Fire        4  721  327.403846   250  680  458.076923  38  115  69.903846   
Flying    641  715  677.750000   245  580  485.000000  40   85  70.750000   
Ghost      92  711  486.500000   275  680  439.562500  20  150  64.437500   
Grass       1  673  344.871429   180  630  421.142857  30  123  67.271429   
Ground     27  645  356.281250   265  770  437.500000  10  115  73.781250   
Ice       124  713  423.541667   250  580  433.458333  36  110  72.000000   
Normal     16  676  319.173469   190  720  401.683673  30  255  77.275510   
Poison     23  691  251.785714   245  535  399.142857  35  105  67.250000   
Psychic    63  720  380.807018   198  780  475.947368  20  190  70.631579   
Rock       74  719  392.727273   280  700  453.750000  30  123  65.363636   
Steel     208  707  442.851852   300  700  487.703704  40  100  65.222222   
Water       7  693  303.089286   200  770  430.455357  20  170  72.062500   

         Attack  ...    Sp. Def Speed                  Generation      \
            min  ...       mean   min  max        mean        min max   
Type 1           ...                                                    
Bug          10  ...  64.797101     5  160   61.681159          1   6   
Dark         50  ...  69.516129    20  125   76.161290          2   6   
Dragon       50  ...  88.843750    40  120   83.031250          1   6   
Electric     30  ...  73.704545    35  140   84.500000          1   6   
Fairy        20  ...  84.705882    15   99   48.588235          1   6   
Fighting     35  ...  64.703704    25  118   66.074074          1   6   
Fire         30  ...  72.211538    20  126   74.442308          1   6   
Flying       30  ...  72.500000    55  123  102.500000          5   6   
Ghost        30  ...  76.468750    20  130   64.343750          1   6   
Grass        27  ...  70.428571    10  145   61.928571          1   6   
Ground       40  ...  62.750000    10  120   63.906250          1   5   
Ice          30  ...  76.291667    25  110   63.458333          1   6   
Normal        5  ...  63.724490     5  135   71.551020          1   6   
Poison       43  ...  64.392857    25  130   63.571429          1   6   
Psychic      20  ...  86.280702    20  180   81.491228          1   6   
Rock         40  ...  75.477273    10  150   55.909091          1   6   
Steel        24  ...  80.629630    23  110   55.259259          2   6   
Water        10  ...  70.517857    15  122   65.964286          1   6   

                   Legendary                   
              mean       min    max      mean  
Type 1                                         
Bug       3.217391     False  False  0.000000  
Dark      4.032258     False   True  0.064516  
Dragon    3.875000     False   True  0.375000  
Electric  3.272727     False   True  0.090909  
Fairy     4.117647     False   True  0.058824  
Fighting  3.370370     False  False  0.000000  
Fire      3.211538     False   True  0.096154  
Flying    5.500000     False   True  0.500000  
Ghost     4.187500     False   True  0.062500  
Grass     3.357143     False   True  0.042857  
Ground    3.156250     False   True  0.125000  
Ice       3.541667     False   True  0.083333  
Normal    3.051020     False   True  0.020408  
Poison    2.535714     False  False  0.000000  
Psychic   3.385965     False   True  0.24561

We can also use `agg` to compute different agg functions for different columns:

In [91]:
agg_dict = {
    'Attack' : 'mean',
    'Defense': ['min', 'max']
}

In [92]:
pokemon.groupby(['Type 1', 'Legendary'], as_index = False).agg(agg_dict)

Type 1 Legendary      Attack Defense     
                              mean     min  max
0        Bug     False   70.971014      30  230
1       Dark     False   86.862069      30  125
2       Dark      True  110.500000      90   95
3     Dragon     False  103.400000      35  130
4     Dragon      True  126.666667      80  121
5   Electric     False   66.125000      15  115
6   Electric      True   98.750000      70   85
7      Fairy     False   57.187500      28   95
8      Fairy      True  131.000000      95   95
9   Fighting     False   96.777778      30   95
10      Fire     False   82.191489      37  140
11      Fire      True  109.000000      85  120
12    Flying     False   50.000000      35   80
13    Flying      True  107.500000      70   80
14     Ghost     False   71.366667      30  145
15     Ghost      True  110.000000     100  120
16     Grass     False   72.119403      30  131
17     Grass      True   97.666667      72  100
18    Ground     False   88.000000      25  130
19    Ground      True  150.000000      90  160
20       Ice     False   73.227273      15  184
21       Ice      True   67.500000     100  100
22    Normal     False   72.083333       5  126
23    Normal      True  140.000000     110  120
24    Poison     False   74.678571      35  120
25   Psychic     False   54.953488      15  120
26   Psychic      True  122.142857      20  160
27      Rock     False   89.925000      40  168
28      Rock      True  122.250000      90  200
29     Steel     False   92.086957      50  230
30     Steel      True   96.250000     100  150
31     Water     False   72.777778      20  180
32     Water      True  111.250000      90  115

### Challenge 1 (20 minutes)

Let's play around with Pandas on a more intricate dataset: a dataset on wines!

**Challenge 14 from the 21 Day Data Challenge** 

Dot's neighbour said that he only likes wine from Stellenbosch, Bordeaux, and the Okanagan Valley, and that the sulfates can't be that high. The problem is, Dot can't really afford to spend tons of money on the wine. Dot's conditions for searching for wine are: 
1. Sulfates cannot be higher than 0.6. 
2. The price has to be less than  $20. 

Use the above conditions to filter the data for questions **2 and 3** below. 

**Questions:**
1. Where is Stellenbosch, anyway? How many wines from Stellenbosch are there in the *entire dataset*? 
2. *After filtering with the 2 conditions*, what is the average price of wine from the Bordeaux region? 
3. *After filtering with the 2 conditions*, what is the least expensive wine that's of the highest quality from the Okanagan Valley?



**Stretch Question:**
1. What is the average price of wine from Stellenbosch, according to the entire unfiltered dataset? 


**Note: Check the dataset to see if there are missing values; if there are, fill in missing values with the mean.**


In [93]:
# code here
df = pd.read_csv('winequality-red_2.csv')

In [122]:
df.head()

,Unnamed: 0,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,region,price
0,0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,Colchagua Valley,64
1,1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5,Bordeaux,89
2,2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5,La Rjoja,25
3,3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6,Willamette,27
4,4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5,Marlborough,9


In [123]:
df[df['region']=='Stellenbosch']
df['region'].value_counts()

La Rjoja            341
Bordeaux            264
Colchagua Valley    260
Okanagan Valley     256
Willamette          233
Marlborough         210
Stellenbosch         35
Name: region, dtype: int64

In [128]:
filtered_df = df[(df['sulphates'] < 0.6) & (df['price'] < 20)]
filtered_df

,Unnamed: 0,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,region,price
4,4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,Marlborough,9
5,5,7.4,0.660,0.00,1.8,0.075,13.0,40.0,0.99780,3.51,0.56,9.4,5,Okanagan Valley,10
7,7,7.3,0.650,0.00,1.2,0.065,15.0,21.0,0.99460,3.39,0.47,10.0,7,Bordeaux,9
25,25,6.3,0.390,0.16,1.4,0.080,11.0,23.0,0.99550,3.34,0.56,9.3,5,Bordeaux,5
38,38,5.7,1.130,0.09,1.5,0.172,7.0,19.0,0.99400,3.50,0.48,9.8,4,Bordeaux,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1556,1556,6.6,0.880,0.04,2.2,0.066,12.0,20.0,0.99636,3.53,0.56,9.9,5,La Rjoja,17
1558,1558,6.9,0.630,0.33,6.7,0.235,66.0,115.0,0.99787,3.22,0.56,9.5,5,Marlborough,4
1562,1562,7.2,0.695,0.13,2.0,0.076,12.0,20.0,0.99546,3.29,0.54,10.1,5,Bordeaux,8
1574,1574,5.6,0.310,0.78,13.9,0.074,23.0,92.0,0.99677,3.39,0.48,10.5,6,Bordeaux,12


In [132]:
filtered_df[filtered_df['region'] == 'Bordeaux']['price'].mean()

11.235294117647058

In [137]:
ok_df = filtered_df[(filtered_df['region'] =='Okanagan Valley')]
ok_df.sort_values(by = ['quality', 'price'], ascending = [False,True]).head(1)

,Unnamed: 0,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,region,price
1017,1017,8.0,0.18,0.37,0.9,0.049,36.0,109.0,0.99007,2.89,0.44,12.7,6,Okanagan Valley,5


### Challenge 2 (25 minutes)

**Challenge 21 from the 21DDC (Adapted)**

Dot wants to play retro video games with all their new friends! Help them figure out which games would be best.

Questions: 
    
1. What is the top 5 best selling games released before the year 2000.

     -  **Note**: Use Global_Sales
    
    
2. Create a new column called Aggregate_Score, which returns the proportional average between Critic Score and User_Score based on Critic_Count and User_Count. Plot a horizontal bar chart of the top 5 highest rated games by Aggregate_Score, not published by Nintendo before the year 2000. From this bar chart, what is the highest rated game by Aggregate_Score?

    -  **Note**: Critic_Count should be filled with the mean. User_Count should be filled with the median.
    
    
#### In the exercise above, there is some missing values in the dataset. Look up the pandas documentation to figure out how to fill missing values in a column. You will be using the **fillna()** function.   

In [ ]:
# code here

# HINT

**How to create the Aggregate Score Column?**

\begin{equation*}
AggregateScore = \frac{(CriticCount * CriticScore)+(UserCount * UserScore)}{UserCount + CriticCount}
\end{equation*}

**Check Your Column Values**

The Critic_Score column is scored out of 100. The User_Score column is scored out of 10. You will need to modify one of the columns to match the other.

## Documentation

In the meantime, check out pandas the user guide in the [pandas documentation](https://pandas.pydata.org/docs/user_guide/index.html#user-guide).

-------
**Why should I use the documentation?**

On the job as a data scientist or data analyst, more often than not, you may find yourself looking up the documentation of a particular function or plugin you use. Don't worry if there are a few functions you don't know by heart. However, there are just too many to know! An essential skill is to learn how to navigate documentation and understand how to apply the examples to your work. 

--------

Additional resources:

- To learn more about these topics, as well as other topics not covered here (e.g. reshaping, merging, additional subsetting methods, working with text data, etc.) check out [these introductory tutorials](https://pandas.pydata.org/docs/getting_started/index.html#getting-started) from the `pandas` documentation
- To learn more about subsetting your data, check out [this tutorial](https://pandas.pydata.org/docs/getting_started/intro_tutorials/03_subset_data.html#min-tut-03-subset)
- This [pandas cheatsheet](https://pandas.pydata.org/Pandas_Cheat_Sheet.pdf) may also be helpful as a reference.